# Abstract 

I propose a method for tabular learning problems where we have an abundance of data but a small amount of it is labeled based on the TabNet architecture.
I demonstrated that by using semi-supervised learning we can improve the performance of the model in the small labeled set setting and check what ammount of data is enough.
I also demonstrated that curriculum learning improves this by improving the self-supervised step. 

# Introduction 

Tabular data problems are still very prevalent in today's world, especially in big corporations that amass large amounts of data for analysis.

Even though this domain is popular, it's not as widely researched as computer vision, audio etc. For example, there are [papers](https://arxiv.org/abs/1604.07379) using self-supervised learning in CV problems as far back as 2016, while the first known one for Tabular data has been released in August 2019 (TabNet, as we'll see).  

Even though large corporations usually have large amounts of data, in many of their tabular problems they have very few labeled examples as those are very expensive to get. **This problem is very common where I work so naturally I wanted to tackle it so I can use it at work**.

To address the scenario where there isn't an abundance of labeled data, the common approach is to use oversampling methods such as [SMOTE](https://arxiv.org/pdf/1106.1813.pdf). Even though these methods sometimes improve the model's performance, the improvement is usually minor at best.

For these reasons, I wanted to implement a self-supervised approach for Tabular Data by learning the underlying representation and then using the pretrained model with the labeled data we have. 

In this project I wanted to test:  
1. If a model trained in a self-supervised fashion gives better results in the small labeled setting.
1. At what number of samples is self supervision unnecesary (as the paper has shown that it does improve the results).
1. If and how curriculum learning improves the outcome and the `self supervised` step. 

To do so I've implemented a relatively new (Aug 2019) Tabular Data DL model called [Tabnet](#https://arxiv.org/pdf/1908.07442.pdf) which uses sequential attention to choose which features to look at at each step, as well as introduced tabular self-supervision for the first time (although I couldn't find any implementation of the self supervision which is why I wanted to implement it). Furthermore, TabNet also enables interpretability by using the sequential attention. I've yet to implement this feature but plan on doing so as I need it for work.  
I've also taken the time to learn the [fastai framework](https://docs.fast.ai/) (a DL framework implemented using `pytorch`) for this project which helped me decouple the different parts and run experiments with minimum changes.


I've tested this approach on 2 different datasets (I wouldn't done more, but it takes a few hours to run each experiment):
1. Adult Census Income - where the task is to distinguish whether a person's income is above $50,000
1. Forest Cover - classifying the forst cover type from cartographic variables.

# Methods 

### Basis for our model - TabNet 

The TabNet architecture uses an encoder-head architecture. 
The encoder is used to learn a better representation of the features in a sequential manner by using masked attention. It is the focus of the TabNet paper. 
The head (a simple FC block for example) then receives the encoder's output to solve the task at hand (classification / regression / decoding). 

##### Encoder

![image.png](static/encoder.png)
TabNet's encoder works by sequentially calculating masks (using an attention block) to be applied to the features. The masked features are then transformed at each step. Half of the transformed features will be used by the decoder, while the other half will be used by the next step's attention block.

The Encoder is built from 2 basic blocks: 

1. Feature Transformer - multiple stacks of blocks made up of FC, BN, GLUs with residual connections. The first few blocks are usually shared since the input's transformations should be the same across all steps. 
![image.png](static/ft.png)

2. Attentive Transformer - creates the mask. A block consisting of a FC, BN and Sparsemax activation (with an additional prior to make sure that the same features won't be used too many times).
![image.png](static/attentive.png)

##### Head 

A simple layer that consists of adding up all the outputs from the encoder's steps and passing them through a FC layer. 

##### Self Supervised Training 

The self supervised training works by creating a mask `S` and applying it to keep some of the features, and then trying to reconstruct the `1-S` left over features. 
To implement the self supervised training, we need to replace the problem's loss (MSE, CE etc) with a loss that takes the forme into account as well as change the model's head to a decoder. 

1. Decoder - As seen above, we used the proposed architecture of a `Feature Transformer` for each step accompanied by a FC layer and then adding up all the results. 
![image.png](static/decoder.png)

2. Loss - For the loss we used the proposed `Reconstruction Loss` which is similar to MSE/MAE for the reconstructed (non masked) features, as well as adding a regularization term (since they're scaled differently). 
    * B - batch size
    * D - # of features
    * S - The mask tensor 
    

![image.png](static/loss.png)

### Datasets

I've tested this approach on 2 different datasets: 
1. Adult Census Income - 32K rows where the task is to distinguish whether a person's income is above $50,000
1. Forest Cover - classifying the forst cover type from cartographic variables.

### Changes I made 

I've made the following changes and checked how they affect the result as explained in the next section.

#### Self Supervised training 

I've implemented the self supervised training in 2 ways:
1. Create a random mask $S$ where the probability of each mask value is $p$. This creates a varying number of masked features at every iteration. 
1. Curricular learning - Progressively mask more features as the number of epochs grow. The intuition is that the model should learn to reconstruct more and more features as it trains (rather than by random). It is similar to a curricular becoming harder (need to reconstruct more values from less data). 

### My Approach

The changes I've made are as outlined in the previous section. 
In this section I'll specify the approach I took when experimenting and in the next section I'll show the results. 

The steps: 
1. Create a baseline - Train the model on the datasets with the same parameters as in the paper to create baseline scores to compare to later (except the `#iterations`. Theirs totaled $~1k$ epochs takes way too long to train).
1. Test the effect of training the `head` on a small training set, with pretraining in a supervised manner on all the data.
1. Check if curricular learning somehow improves the previous results.


I've also experimented with a few other architectures for the decoder, classifier head. However, I didn't run exhaustive tests as the tests I ran took ~60 hours to run. So I'll just say that I didn't notice any noticeable results but I suspect that running for more epochs might do the trick (as I found it mattered a lot here). 

# Experiments

The experiments for self-supervision can be found in the `self_supervision` notebooks

### Baseline 

First I needed to create a baseline to compare results to. To create that baseline I implemented the TabNet model and ran it on the Adult, Forest Cover datasets multiple times with the parameters that were reported in the paper. 

I trained for 350 epochs VS 1K in the paper to reduce runtime, with different validation set sizes as can be seen in the following table (which includes the resulting accuracy): 
![image.png](static/table.jpg)

Each training took ~36 min (18 before SS, 18 after), SS phase took ~10 min. Overall I did 124 runs as you'll see in the discussions below. 

#### Adult Dataset 

I trained each model with differing validation set sizes as outlined above, while using the Adam optimizer and the [1-cycle training policy](https://arxiv.org/abs/1708.07120) for setting the LR (with a min LR of $5e^{-3}$, max LR of $1e{-2}$). 

I set the hyperparameters as reported in the paper as following: $N_d=N_a=64$, $\lambda_{sparse}=1e{-4}$, $batch size=16K$, 
$virtual batch size=128$, $N_{steps}=5$, $\gamma=1.5$ with 2 shared blocks, 2 independent ones per `FeatureTransformer`. 
I trained the `decoder` with the paper's architecture and with $N_{steps}=10$. 

#### Forest Cover Dataset 

Same as before but I set the hyperparameters as reported in the paper as following: $N_d=N_a=16$, $\lambda_{sparse}=1e{-4}$, $batch size=4096$, 
$virtual batch size=128$, $N_{steps}=5$, $\gamma=1.5$ with 2 shared blocks, 2 independent ones per `FeatureTransformer`. 


# Discussions


As explained before, we've run the experiments on different validation set sizes (which correspond to different train set sizes) to check the effects of self supervision.

In this section we'll discuss the results listed in the table 

#### The effects of Self Supervision 
The following graph depicts the scores with the stddev VS the size of the validation set for the Forest Cover dataset. As we can see the score goes as the validation set % grows (i.e. training set size decreases). In the last experiment for example, we use a validation set of 99.5% which leaves 0.5% (~160 samples) for training. 

Furthermore, we can see that overall self supervision improved both the mean accuracy, as well as the std across different runs.
Notice for example that for a 20% validation set (80-20 split), the stddev was 0.5% after SS as opposed to 15% before. 
For later runs, both the result and stddev gap seems to diminish, although I suspect that might be because for small training sets we need more epochs for the self-supervision and final training as I suspect the model can achieve better scores.

![image.png](static/forest_before.jpg)

For the Adult Census dataset however we didn't notice any improvment: 
![image.png](static/adult_before.png)

#### After Curriculur Learning 

The following graph depicts the scores with the stddev VS the size of the validation set for the Forest Cover dataset after self supervision, with and without curicular learning. 
When I ran the experiments I witnessed a noticeable reduction in the validation set loss, however, as we can see, there isn't any noticeable improvment in the accuracy or the stddev of the results. 

Similar results are given for the Adult Census dataset. 

![image.png](static/forest_after.png)

#### Conclusions 

It seems to me that while the intuition that large dataset with many features can benefit from SS is solid it doesn't give as much of a boost as I'd have hoped (which is consistent with the paper's results).

Furthermore, as the experiments showed, using a "smarter" training regime such as Curricular Learning doesn't seem to have any effect on the self supervision and final training results. 